In [1]:
from bottle import Bottle, route, run
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint


url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest'
parameters = {
  'slug':'bitcoin',
  'convert':'PLN'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '0338f88c-4dc5-4997-b9dd-a7dd973e4a0d',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  pprint.pprint(json.loads(response.text)['data']['1']['quote']['USD']['price'])
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

app = Bottle()

@app.route('/index')
def hello():
    return '<h1>Hello</h1>'

run(app, host = 'localhost', port = 8080, debug = True)

48819.75913664301


Bottle v0.12.19 server starting up (using WSGIRefServer())...
Listening on http://localhost:8080/
Hit Ctrl-C to quit.

C:\Users\leski\anaconda3\lib\site-packages\bottle.py:3139: ResourceWarning: unclosed <socket.socket fd=1348, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('127.0.0.1', 8080)>
  pass


In [100]:
#commit Jarek 22.12
#Poprawilem api, trzeba po innym linku sie laczyc aby seryjnie zaciagac 

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pprint

import pandas as pd

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'

currency = 'USD'

parameters = {
  'convert': currency
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '25bdeaa9-d947-4b43-83b5-f2ee7bb97197',
}

session = Session()
session.headers.update(headers)
coins_names = []
coins_prices = []
coins_volume_24h = []
coins_market_cap = []
coins_shortcut = []
coins_id = []

try:
    response = session.get(url, params=parameters)
    
except (ConnectionError, Timeout, TooManyRedirects) as e:
    print(e)
    
for coins in range(100):
    coins_names.append((json.loads(response.text)['data'][coins]['name']))
    coins_prices.append((json.loads(response.text)['data'][coins]['quote'][currency]['price']))
    coins_volume_24h.append((json.loads(response.text)['data'][coins]['quote'][currency]['volume_24h']))
    coins_market_cap.append((json.loads(response.text)['data'][coins]['quote'][currency]['market_cap']))
    coins_shortcut.append((json.loads(response.text)['data'][coins]['symbol']))
    coins_id.append((json.loads(response.text)['data'][coins]['id']))
                          
rtime = json.loads(response.text)['data'][1]["last_updated"]
data = list(zip(coins_names,coins_prices, coins_volume_24h, coins_market_cap))
dataframe = pd.DataFrame(data, columns = ['Crypto', 'Price', 'volume 24h', 'Market Cap'])

print("Update time:", rtime)
print(coins_shortcut)
print(dataframe)



Update time: 2021-12-22T23:12:02.000Z
['BTC', 'ETH', 'BNB', 'USDT', 'SOL', 'XRP', 'ADA', 'USDC', 'LUNA', 'AVAX', 'DOT', 'DOGE', 'SHIB', 'MATIC', 'BUSD', 'CRO', 'WBTC', 'LTC', 'UNI', 'LINK', 'UST', 'DAI', 'ALGO', 'BCH', 'TRX', 'XLM', 'NEAR', 'ATOM', 'AXS', 'MANA', 'HBAR', 'FTT', 'VET', 'BTCB', 'FIL', 'EGLD', 'ETC', 'SAND', 'ICP', 'THETA', 'HNT', 'FTM', 'XTZ', 'MIOTA', 'XMR', 'LEO', 'GRT', 'KLAY', 'EOS', 'GALA', 'LRC', 'CAKE', 'STX', 'ONE', 'AAVE', 'FLOW', 'BTT', 'MKR', 'BSV', 'KSM', 'CRV', 'ENJ', 'QNT', 'ZEC', 'AMP', 'XEC', 'NEO', 'OKB', 'RUNE', 'BAT', 'AR', 'KDA', 'CELO', 'KCS', 'WAVES', 'CHZ', 'HT', 'DASH', 'HOT', 'COMP', 'NEXO', 'TUSD', 'XEM', 'IOTX', 'YFI', 'MINA', 'TFUEL', '1INCH', 'XDC', 'ICX', 'ROSE', 'DCR', 'USDP', 'QTUM', 'CEL', 'OMG', 'BORA', 'RVN', 'ANKR', 'WAXP']
          Crypto         Price    volume 24h    Market Cap
0        Bitcoin  48808.566354  2.439748e+10  9.228806e+11
1       Ethereum   3990.915258  1.396719e+10  4.743178e+11
2   Binance Coin    537.636201  1.5872

In [102]:
#commit Jarek 23.11 
#Dodane API z yahoo do zaciagania danych historycznych 
#dla kazdego symbolu krypto zaciagnietego z CMC pobiera dane historyczne z yahoo, ktore zapisuje do listy dfów 'historical'
#jak wpadna nowe rekordy (bedzie nowy dzien) pobierze tylko te notowania, ktorych mu brakuje i zconcatuje do aktualnego df 
#mozna wybrac walute, ale yahoo nie obsluguje PLN -> USD, EUR smiga 
 
import pandas_datareader as pdr
from datetime import date
import json 
import requests


today = date.today().strftime("%Y,%m,%d")
parts = today.split(",")
year_e, month_e, day_e = int(parts[0]), int(parts[1]), int(parts[2])
#historical = []
ac_historical = []
x = 0


for coin in coins_shortcut:
    
    try:
    
        y = historical[x].sort_values(by = 'Date', ascending = True).tail(1)
        year_s, month_s, day_s  = (y['Date']).dt.year.values[0], (y['Date']).dt.month.values[0], (y['Date']).dt.day.values[0]
        start = dt.datetime(year_s,month_s,day_s)
        end = dt.datetime(year_e,month_e,day_e)
        df = ac_historical
        
    except:
        
        start = dt.datetime(2000,1,1)
        end = dt.datetime(year_e,month_e,day_e)
        df = historical
        
    try:
        tag = coin + '-' + currency
        df.append(pdr.get_data_yahoo(tag, start , end ))
        df[x]['Crypto'] = coin
        df[x].reset_index(inplace = True, drop = False)
        
    except:

        try:
            tag = coin + '1-' + currency
            df.append(pdr.get_data_yahoo(tag, start , end ))
            df[x]['Crypto'] = coin
            df[x].reset_index(inplace = True, drop = False)
        except:

            try:
                tag = coin + '3-' + currency
                df.append(pdr.get_data_yahoo(tag, start , end ))
                df[x]['Crypto'] = coin
                df[x].reset_index(inplace = True, drop = False)
            except:
                pass
    x += 1
    


historical_df = pd.concat(historical)

try:
    ac_historical_df = pd.concat(ac_historical)
    historical = [historical_df, ac_historical_df]
    historical_base = pd.concat(historical)
    historical_base.reset_index(inplace = True, drop = True)
    
except: 
    historical_base = historical_df
    historical_base.reset_index(inplace = True, drop = True)

historical_base.tail(20)
   



,Date,High,Low,Open,Close,Volume,Adj Close,Crypto
230013,2021-12-05,0.521253,0.449738,0.517357,0.466703,7.786045e+07,0.466703,WAXP
230014,2021-12-06,0.498671,0.404777,0.467531,0.485017,1.229421e+08,0.485017,WAXP
230015,2021-12-07,0.510557,0.472023,0.486144,0.485799,8.448266e+07,0.485799,WAXP
230016,2021-12-08,0.728246,0.456406,0.485070,0.663711,8.467941e+08,0.663711,WAXP
230017,2021-12-09,0.711925,0.562528,0.673510,0.563387,6.067911e+08,0.563387,WAXP
230018,2021-12-10,0.587127,0.505400,0.565957,0.521420,2.792368e+08,0.521420,WAXP
230019,2021-12-11,0.540171,0.502898,0.524753,0.535177,1.052898e+08,0.535177,WAXP
230020,2021-12-12,0.535306,0.508792,0.535306,0.527960,6.139032e+07,0.527960,WAXP
230021,2021-12-13,0.531475,0.437049,0.528968,0.446578,9.248520e+07,0.446578,WAXP
230022,2021-12-14,0.448329,0.424490,0.445093,0.424490,7.862487e+07,0.424490,WAXP
